## 1. Setup + Imports

In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.express as px

ROOT = Path.cwd().parent
DATA = ROOT / "data" / "processed" / "analytics_table.parquet"

FIGS = ROOT / "reports" / "figures"
FIGS.mkdir(parents=True, exist_ok=True)

def save_fig(fig, path: Path, *, scale: int = 2) -> None:
    path.parent.mkdir(parents=True, exist_ok=True)
    fig.write_image(str(path), scale=scale)

## 2. Load processed data + Quick audit 


In [15]:
df = pd.read_parquet(DATA)

print("rows:", len(df), "cols:", len(df.columns))
print(df.dtypes.head(15))

missing = df.isna().sum().sort_values(ascending=False).head(10)
print(missing)

rows: 100 cols: 18
order_id              string[python]
user_id               string[python]
amount                       Float64
quantity                       Int64
created_at       datetime64[ns, UTC]
status                        object
status_clean                  object
amount_isna                     bool
quantity_isna                   bool
date                          object
year                         float64
month                 string[python]
dow                           object
hour                         float64
country                       object
dtype: object
amount_is_outlier    12
amount               12
amount_winsor        12
quantity              8
year                  7
hour                  7
dow                   7
month                 7
date                  7
created_at            7
dtype: int64


**Dataset quality notes:**

- The dataset contains 100 rows and 18 columns, indicating a reasonably sized analytical table.
- Missing values are concentrated in a small number of columns, which may require imputation or exclusion.


## 3. Questions + results

### Questions

1. How does total revenue, number of orders, and average order value (AOV) vary by country?
2. Does daily revenue show a consistent pattern or high variability over time?
3. What does the distribution of winsorized order amounts tell us about a typical order and the presence of skew or outliers?
4. Does increasing order quantity lead to higher order amounts?

#### 1. How does total revenue, number of orders (n), and average order value (AOV) vary by country?

In [16]:
rev = (
    df.groupby("country", dropna=False)
    .agg(
        n=("order_id", "size"),
        revenue=("amount", "sum"),
        aov=("amount", "mean"),
    )
    .reset_index()
    .sort_values("revenue", ascending=False)
)

fig = px.bar(
    rev,
    x="country",
    y="revenue",
    title="Revenue by country (all data)"
)

fig.update_layout(title={"x": 0.02})
fig.update_xaxes(title_text="Country")
fig.update_yaxes(title_text="Revenue (sum of amount)")
fig.show()

save_fig(fig, FIGS / "revenue_by_country.png")


#### 1.1 Interpretations

- Saudi Arabia (SA) generates substantially higher total revenue than the UAE (AE), indicating that SA is the primary contributor to overall revenue in this dataset.
- The higher revenue in SA is likely due to more orders, higher average order values, or both, compared to AE.
- AE generates less revenue than SA, but it still contributes to total revenue and could grow if order volume or spending increases.

#### 1.2 Caveat

This comparison is based on total revenue only and does not account for differences in population size, number of customers, or time coverage between countries.

#### 2. Does daily revenue show a consistent pattern or high variability over time?

In [25]:
trend = (
    df.groupby("date", dropna=False)
    .agg(
        n=("order_id", "size"),
        revenue=("amount", "sum")
    )
    .reset_index()
    .sort_values("date")
)

fig = px.line(
    trend,
    x="date",
    y="revenue",
    title="Revenue over time (daily)"
)

fig.update_layout(title={"x": 0.02})
fig.update_xaxes(title_text="date")
fig.update_yaxes(title_text="Revenue")
fig.show()

save_fig(fig, FIGS / "revenue_trend_daily.png")

#### 2.1 Interpretations

- Daily revenue shows high variability over time, with frequent change rather than a consistent pattern.
- Revenue spikes occur on certain days, indicating irregular or one-off events instead of steady daily performance.
- There is no clear, stable daily trend, suggesting revenue changes significantly from day to day.

#### 2.2 Caveat

Daily revenue data is noisy, so short-term variability may hide longer-term trends.

### 3. What does the distribution of winsorized order amounts tell us about a typical order and the presence of skew or outliers?

In [26]:
fig = px.histogram(
    df,
    x="amount_winsor",
    nbins=30,
    title="Order amount distribution (winsorized)"
)

fig.update_layout(title={"x": 0.02})
fig.update_xaxes(title_text="Amount (winsorized)")
fig.update_yaxes(title_text="Number of orders")
fig.show()

save_fig(fig, FIGS / "amount_hist_winsor.png")

#### 3.1 Interpretations

- Most orders have relatively low amounts, indicating that a typical order falls in the lower price range.
- The distribution is right-skewed, with a small number of higher-value orders extending to the right tail.
- High-amount orders are less frequent, even after winsorization, suggesting they are uncommon compared to typical orders.

#### 3.2 Caveat

Winsorization caps extreme values, so the true magnitude of the highest order amounts is not fully visible in this chart.

### 4. Does increasing order quantity lead to higher order amounts?

In [27]:
fig = px.scatter(
    df,
    x="quantity",
    y="amount",
    title="Relationship between order quantity and order amount"
)

fig.update_layout(title={"x": 0.02})
fig.update_xaxes(title_text="Quantity")
fig.update_yaxes(title_text="Order amount")
fig.show()

save_fig(fig, FIGS / "quantity_vs_amount_scatter.png")

#### 4.1 Interpretations

- Higher quantities generally correspond to higher order amounts, indicating a positive relationship between quantity and spending.
- Most orders cluster at low quantities and lower amounts, suggesting that typical purchases involve small quantities.
- Some points lie far from the main cluster, showing occasional large orders that differ from typical buying behavior.


#### 3.2 Caveat

This plot shows correlation, not causation, and extreme values or pricing differences per item may influence the relationship.

## 4. Bootstrap comparison

In [31]:
import numpy as np
import pandas as pd

def bootstrap_diff_means(
    a: pd.Series,
    b: pd.Series,
    *,
    n_boot: int = 2000,
    seed: int = 0
) -> dict:
    rng = np.random.default_rng(seed)

    a = pd.to_numeric(a, errors="coerce").dropna().to_numpy()
    b = pd.to_numeric(b, errors="coerce").dropna().to_numpy()
    assert len(a) > 0 and len(b) > 0, "Empty group after cleaning"

    diffs = []
    for _ in range(n_boot):
        sa = rng.choice(a, size=len(a), replace=True)
        sb = rng.choice(b, size=len(b), replace=True)
        diffs.append(sa.mean() - sb.mean())

    diffs = np.array(diffs)

    return {
        "diff_mean": float(a.mean() - b.mean()),
        "ci_low": float(np.quantile(diffs, 0.025)),
        "ci_high": float(np.quantile(diffs, 0.975)),
    }


d = df.assign(is_refund=df["status_clean"].eq("refund").astype(int))

a = d.loc[d["country"].eq("SA"), "is_refund"]
b = d.loc[d["country"].eq("AE"), "is_refund"]

print("n_SA:", len(a), "n_AE:", len(b))
print(bootstrap_diff_means(a, b, n_boot=2000, seed=0))


n_SA: 76 n_AE: 24
{'diff_mean': -0.0899122807017544, 'ci_low': -0.2807017543859649, 'ci_high': 0.07461622807017514}


### Interpretation

- The estimated refund rate in Saudi Arabia is slightly lower than in the UAE, but the size of this difference is small and uncertain based on the bootstrap results.
- Because the 95% confidence interval includes zero, the data does not provide strong evidence that refund rates differ meaningfully between the two countries.

## 5. Findings

The dataset contains 100 records with limited missing values in a few numeric and time-related fields. Saudi Arabia generates substantially higher revenue than the UAE. Revenue over time is volatile with no clear trend, while most orders are low in value with a right-skewed distribution. Order amount generally increases with quantity, and refund rates between SA and AE do not differ meaningfully.